In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import string
import math
import time

ALL_CHARS = string.ascii_lowercase + ".,;'"
N_CHARS = len(ALL_CHARS)
EOS = 'EOS'

data = [
    "Alice", "Bob", "Charlie", "David", "Eve", "Fiona", "George", 
    "Hannah", "Ivan", "Julia", "Kevin", "Laura", "Mike", "Nancy", 
    "Oscar", "Peter", "Quinn", "Rachel", "Steve", "Tina", "Ursula", 
    "Victor", "Wendy", "Xavier", "Yara", "Zoe"
]

In [ ]:
def char_to_index(char):
    return ALL_CHARS.index(char)

def char_to_tensor(char):
    tensor = torch.zeros((1, N_CHARS))
    tensor[0][char_to_index(char)] = 1
    return tensor

def name_to_tensor(name):
    tensor = torch.zeros((len(name), 1, N_CHARS))
    for i, char in enumerate(name):
        tensor[i][0][char_to_index(char.lower())] = 1
    return tensor

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        
        # Input to hidden
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        # Input to output
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, input, hidden):
        # Combine input and hidden state
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)

# Initialize model
n_hidden = 128
rnn = RNN(N_CHARS, n_hidden, N_CHARS)

# Loss and optimizer
criterion = nn.NLLLoss()
learning_rate = 0.005
optimizer = optim.SGD(rnn.parameters(), lr=learning_rate)